In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch
import gc

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
!pip install ../input/openaiclipweights/python-ftfy-master/python-ftfy-master

In [ ]:
!pip install ../input/openaiclipweights/clip/CLIP

In [ ]:
!ls ../input/openaiclipweights/clip/CLIP/models

In [ ]:
!cp ../input/openaiclipweights/CLIP-main/CLIP-main/clip/bpe_simple_vocab_16e6.txt /opt/conda/lib/python3.7/site-packages/clip/.
!gzip -k /opt/conda/lib/python3.7/site-packages/clip/bpe_simple_vocab_16e6.txt
!ls /opt/conda/lib/python3.7/site-packages/clip/.

In [ ]:
import torch
import clip
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, Dataset

print("Torch version:", torch.__version__)

In [ ]:
train = pd.read_csv('../input/petfinderdata/train-folds-1.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
sub = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
#train = train.sort_values('Pawpularity').reset_index(drop=True)
#train['fold'] = np.arange(train.shape[0]) % 10
BS = 32
print(train.shape, test.shape, sub.shape)

In [ ]:
clip.available_models()

In [ ]:
import joblib

class CustomDataset:
    def __init__(self, data, base_path='../input/petfinder-pawpularity-score/test/', preprocess=None):
        
        self.data = data.copy()
        self.base_path = base_path
        if 'Pawpularity' not in self.data.columns:
            self.data['Pawpularity'] = 0
        self.preprocess=preprocess
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        img = Image.open(self.base_path + self.data.Id[item] + '.jpg').convert("RGB")
        img = self.preprocess(img)
        return img

RES = {}
for m in ['RN50', 'RN101', 'RN50x4', 'RN50x16', 'ViT-B-16', 'ViT-B-32']:
    model, preprocess = clip.load("../input/openaiclipweights/clip/CLIP/models/"+m+".pt")
    model.cuda().eval()
    EMB = []
    with torch.no_grad():
        test_dataset = CustomDataset(data = train, base_path='../input/petfinder-pawpularity-score/train/', preprocess=preprocess)
        test_data_loader = DataLoader(test_dataset, batch_size=64,num_workers=2,shuffle=False,pin_memory=True,)
        for batch in tqdm(test_data_loader):
            image_features = model.encode_image(batch.to('cuda'))
            #image_features /= image_features.norm(dim=-1, keepdim=True)
            logits = image_features.cpu().numpy()
            EMB.append(logits)
    EMB = np.concatenate(EMB, 0)
    EMB = EMB.astype('float32')
    gc.collect()
    RES['clip_'+m] = EMB

    del model
    torch.cuda.empty_cache()
    gc.collect()

joblib.dump(RES, 'train-embeddings-openai-clip-1.joblib')

In [ ]:
# model0.cuda().eval()
# model1.cuda().eval()
# input_resolution = model0.visual.input_resolution
# context_length = model0.context_length
# vocab_size = model0.vocab_size

# print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model0.parameters()]):,}")
# print("Input resolution:", input_resolution)
# print("Context length:", context_length)
# print("Vocab size:", vocab_size)

In [ ]:
# #imagenet_classes = ["tench", "goldfish", "great white shark", "tiger shark", "hammerhead shark", "electric ray", "stingray", "rooster", "hen", "ostrich", "brambling", "goldfinch", "house finch", "junco", "indigo bunting", "American robin", "bulbul", "jay", "magpie", "chickadee", "American dipper", "kite (bird of prey)", "bald eagle", "vulture", "great grey owl", "fire salamander", "smooth newt", "newt", "spotted salamander", "axolotl", "American bullfrog", "tree frog", "tailed frog", "loggerhead sea turtle", "leatherback sea turtle", "mud turtle", "terrapin", "box turtle", "banded gecko", "green iguana", "Carolina anole", "desert grassland whiptail lizard", "agama", "frilled-necked lizard", "alligator lizard", "Gila monster", "European green lizard", "chameleon", "Komodo dragon", "Nile crocodile", "American alligator", "triceratops", "worm snake", "ring-necked snake", "eastern hog-nosed snake", "smooth green snake", "kingsnake", "garter snake", "water snake", "vine snake", "night snake", "boa constrictor", "African rock python", "Indian cobra", "green mamba", "sea snake", "Saharan horned viper", "eastern diamondback rattlesnake", "sidewinder rattlesnake", "trilobite", "harvestman", "scorpion", "yellow garden spider", "barn spider", "European garden spider", "southern black widow", "tarantula", "wolf spider", "tick", "centipede", "black grouse", "ptarmigan", "ruffed grouse", "prairie grouse", "peafowl", "quail", "partridge", "african grey parrot", "macaw", "sulphur-crested cockatoo", "lorikeet", "coucal", "bee eater", "hornbill", "hummingbird", "jacamar", "toucan", "duck", "red-breasted merganser", "goose", "black swan", "tusker", "echidna", "platypus", "wallaby", "koala", "wombat", "jellyfish", "sea anemone", "brain coral", "flatworm", "nematode", "conch", "snail", "slug", "sea slug", "chiton", "chambered nautilus", "Dungeness crab", "rock crab", "fiddler crab", "red king crab", "American lobster", "spiny lobster", "crayfish", "hermit crab", "isopod", "white stork", "black stork", "spoonbill", "flamingo", "little blue heron", "great egret", "bittern bird", "crane bird", "limpkin", "common gallinule", "American coot", "bustard", "ruddy turnstone", "dunlin", "common redshank", "dowitcher", "oystercatcher", "pelican", "king penguin", "albatross", "grey whale", "killer whale", "dugong", "sea lion", "Chihuahua", "Japanese Chin", "Maltese", "Pekingese", "Shih Tzu", "King Charles Spaniel", "Papillon", "toy terrier", "Rhodesian Ridgeback", "Afghan Hound", "Basset Hound", "Beagle", "Bloodhound", "Bluetick Coonhound", "Black and Tan Coonhound", "Treeing Walker Coonhound", "English foxhound", "Redbone Coonhound", "borzoi", "Irish Wolfhound", "Italian Greyhound", "Whippet", "Ibizan Hound", "Norwegian Elkhound", "Otterhound", "Saluki", "Scottish Deerhound", "Weimaraner", "Staffordshire Bull Terrier", "American Staffordshire Terrier", "Bedlington Terrier", "Border Terrier", "Kerry Blue Terrier", "Irish Terrier", "Norfolk Terrier", "Norwich Terrier", "Yorkshire Terrier", "Wire Fox Terrier", "Lakeland Terrier", "Sealyham Terrier", "Airedale Terrier", "Cairn Terrier", "Australian Terrier", "Dandie Dinmont Terrier", "Boston Terrier", "Miniature Schnauzer", "Giant Schnauzer", "Standard Schnauzer", "Scottish Terrier", "Tibetan Terrier", "Australian Silky Terrier", "Soft-coated Wheaten Terrier", "West Highland White Terrier", "Lhasa Apso", "Flat-Coated Retriever", "Curly-coated Retriever", "Golden Retriever", "Labrador Retriever", "Chesapeake Bay Retriever", "German Shorthaired Pointer", "Vizsla", "English Setter", "Irish Setter", "Gordon Setter", "Brittany dog", "Clumber Spaniel", "English Springer Spaniel", "Welsh Springer Spaniel", "Cocker Spaniel", "Sussex Spaniel", "Irish Water Spaniel", "Kuvasz", "Schipperke", "Groenendael dog", "Malinois", "Briard", "Australian Kelpie", "Komondor", "Old English Sheepdog", "Shetland Sheepdog", "collie", "Border Collie", "Bouvier des Flandres dog", "Rottweiler", "German Shepherd Dog", "Dobermann", "Miniature Pinscher", "Greater Swiss Mountain Dog", "Bernese Mountain Dog", "Appenzeller Sennenhund", "Entlebucher Sennenhund", "Boxer", "Bullmastiff", "Tibetan Mastiff", "French Bulldog", "Great Dane", "St. Bernard", "husky", "Alaskan Malamute", "Siberian Husky", "Dalmatian", "Affenpinscher", "Basenji", "pug", "Leonberger", "Newfoundland dog", "Great Pyrenees dog", "Samoyed", "Pomeranian", "Chow Chow", "Keeshond", "brussels griffon", "Pembroke Welsh Corgi", "Cardigan Welsh Corgi", "Toy Poodle", "Miniature Poodle", "Standard Poodle", "Mexican hairless dog (xoloitzcuintli)", "grey wolf", "Alaskan tundra wolf", "red wolf or maned wolf", "coyote", "dingo", "dhole", "African wild dog", "hyena", "red fox", "kit fox", "Arctic fox", "grey fox", "tabby cat", "tiger cat", "Persian cat", "Siamese cat", "Egyptian Mau", "cougar", "lynx", "leopard", "snow leopard", "jaguar", "lion", "tiger", "cheetah", "brown bear", "American black bear", "polar bear", "sloth bear", "mongoose", "meerkat", "tiger beetle", "ladybug", "ground beetle", "longhorn beetle", "leaf beetle", "dung beetle", "rhinoceros beetle", "weevil", "fly", "bee", "ant", "grasshopper", "cricket insect", "stick insect", "cockroach", "praying mantis", "cicada", "leafhopper", "lacewing", "dragonfly", "damselfly", "red admiral butterfly", "ringlet butterfly", "monarch butterfly", "small white butterfly", "sulphur butterfly", "gossamer-winged butterfly", "starfish", "sea urchin", "sea cucumber", "cottontail rabbit", "hare", "Angora rabbit", "hamster", "porcupine", "fox squirrel", "marmot", "beaver", "guinea pig", "common sorrel horse", "zebra", "pig", "wild boar", "warthog", "hippopotamus", "ox", "water buffalo", "bison", "ram (adult male sheep)", "bighorn sheep", "Alpine ibex", "hartebeest", "impala (antelope)", "gazelle", "arabian camel", "llama", "weasel", "mink", "European polecat", "black-footed ferret", "otter", "skunk", "badger", "armadillo", "three-toed sloth", "orangutan", "gorilla", "chimpanzee", "gibbon", "siamang", "guenon", "patas monkey", "baboon", "macaque", "langur", "black-and-white colobus", "proboscis monkey", "marmoset", "white-headed capuchin", "howler monkey", "titi monkey", "Geoffroy's spider monkey", "common squirrel monkey", "ring-tailed lemur", "indri", "Asian elephant", "African bush elephant", "red panda", "giant panda", "snoek fish", "eel", "silver salmon", "rock beauty fish", "clownfish", "sturgeon", "gar fish", "lionfish", "pufferfish", "abacus", "abaya", "academic gown", "accordion", "acoustic guitar", "aircraft carrier", "airliner", "airship", "altar", "ambulance", "amphibious vehicle", "analog clock", "apiary", "apron", "trash can", "assault rifle", "backpack", "bakery", "balance beam", "balloon", "ballpoint pen", "Band-Aid", "banjo", "baluster / handrail", "barbell", "barber chair", "barbershop", "barn", "barometer", "barrel", "wheelbarrow", "baseball", "basketball", "bassinet", "bassoon", "swimming cap", "bath towel", "bathtub", "station wagon", "lighthouse", "beaker", "military hat (bearskin or shako)", "beer bottle", "beer glass", "bell tower", "baby bib", "tandem bicycle", "bikini", "ring binder", "binoculars", "birdhouse", "boathouse", "bobsleigh", "bolo tie", "poke bonnet", "bookcase", "bookstore", "bottle cap", "hunting bow", "bow tie", "brass memorial plaque", "bra", "breakwater", "breastplate", "broom", "bucket", "buckle", "bulletproof vest", "high-speed train", "butcher shop", "taxicab", "cauldron", "candle", "cannon", "canoe", "can opener", "cardigan", "car mirror", "carousel", "tool kit", "cardboard box / carton", "car wheel", "automated teller machine", "cassette", "cassette player", "castle", "catamaran", "CD player", "cello", "mobile phone", "chain", "chain-link fence", "chain mail", "chainsaw", "storage chest", "chiffonier", "bell or wind chime", "china cabinet", "Christmas stocking", "church", "movie theater", "cleaver", "cliff dwelling", "cloak", "clogs", "cocktail shaker", "coffee mug", "coffeemaker", "spiral or coil", "combination lock", "computer keyboard", "candy store", "container ship", "convertible", "corkscrew", "cornet", "cowboy boot", "cowboy hat", "cradle", "construction crane", "crash helmet", "crate", "infant bed", "Crock Pot", "croquet ball", "crutch", "cuirass", "dam", "desk", "desktop computer", "rotary dial telephone", "diaper", "digital clock", "digital watch", "dining table", "dishcloth", "dishwasher", "disc brake", "dock", "dog sled", "dome", "doormat", "drilling rig", "drum", "drumstick", "dumbbell", "Dutch oven", "electric fan", "electric guitar", "electric locomotive", "entertainment center", "envelope", "espresso machine", "face powder", "feather boa", "filing cabinet", "fireboat", "fire truck", "fire screen", "flagpole", "flute", "folding chair", "football helmet", "forklift", "fountain", "fountain pen", "four-poster bed", "freight car", "French horn", "frying pan", "fur coat", "garbage truck", "gas mask or respirator", "gas pump", "goblet", "go-kart", "golf ball", "golf cart", "gondola", "gong", "gown", "grand piano", "greenhouse", "radiator grille", "grocery store", "guillotine", "hair clip", "hair spray", "half-track", "hammer", "hamper", "hair dryer", "hand-held computer", "handkerchief", "hard disk drive", "harmonica", "harp", "combine harvester", "hatchet", "holster", "home theater", "honeycomb", "hook", "hoop skirt", "gymnastic horizontal bar", "horse-drawn vehicle", "hourglass", "iPod", "clothes iron", "carved pumpkin", "jeans", "jeep", "T-shirt", "jigsaw puzzle", "rickshaw", "joystick", "kimono", "knee pad", "knot", "lab coat", "ladle", "lampshade", "laptop computer", "lawn mower", "lens cap", "letter opener", "library", "lifeboat", "lighter", "limousine", "ocean liner", "lipstick", "slip-on shoe", "lotion", "music speaker", "loupe magnifying glass", "sawmill", "magnetic compass", "messenger bag", "mailbox", "tights", "one-piece bathing suit", "manhole cover", "maraca", "marimba", "mask", "matchstick", "maypole", "maze", "measuring cup", "medicine cabinet", "megalith", "microphone", "microwave oven", "military uniform", "milk can", "minibus", "miniskirt", "minivan", "missile", "mitten", "mixing bowl", "mobile home", "ford model t", "modem", "monastery", "monitor", "moped", "mortar and pestle", "graduation cap", "mosque", "mosquito net", "vespa", "mountain bike", "tent", "computer mouse", "mousetrap", "moving van", "muzzle", "metal nail", "neck brace", "necklace", "baby pacifier", "notebook computer", "obelisk", "oboe", "ocarina", "odometer", "oil filter", "pipe organ", "oscilloscope", "overskirt", "bullock cart", "oxygen mask", "product packet / packaging", "paddle", "paddle wheel", "padlock", "paintbrush", "pajamas", "palace", "pan flute", "paper towel", "parachute", "parallel bars", "park bench", "parking meter", "railroad car", "patio", "payphone", "pedestal", "pencil case", "pencil sharpener", "perfume", "Petri dish", "photocopier", "plectrum", "Pickelhaube", "picket fence", "pickup truck", "pier", "piggy bank", "pill bottle", "pillow", "ping-pong ball", "pinwheel", "pirate ship", "drink pitcher", "block plane", "planetarium", "plastic bag", "plate rack", "farm plow", "plunger", "Polaroid camera", "pole", "police van", "poncho", "pool table", "soda bottle", "plant pot", "potter's wheel", "power drill", "prayer rug", "printer", "prison", "missile", "projector", "hockey puck", "punching bag", "purse", "quill", "quilt", "race car", "racket", "radiator", "radio", "radio telescope", "rain barrel", "recreational vehicle", "fishing casting reel", "reflex camera", "refrigerator", "remote control", "restaurant", "revolver", "rifle", "rocking chair", "rotisserie", "eraser", "rugby ball", "ruler measuring stick", "sneaker", "safe", "safety pin", "salt shaker", "sandal", "sarong", "saxophone", "scabbard", "weighing scale", "school bus", "schooner", "scoreboard", "CRT monitor", "screw", "screwdriver", "seat belt", "sewing machine", "shield", "shoe store", "shoji screen / room divider", "shopping basket", "shopping cart", "shovel", "shower cap", "shower curtain", "ski", "balaclava ski mask", "sleeping bag", "slide rule", "sliding door", "slot machine", "snorkel", "snowmobile", "snowplow", "soap dispenser", "soccer ball", "sock", "solar thermal collector", "sombrero", "soup bowl", "keyboard space bar", "space heater", "space shuttle", "spatula", "motorboat", "spider web", "spindle", "sports car", "spotlight", "stage", "steam locomotive", "through arch bridge", "steel drum", "stethoscope", "scarf", "stone wall", "stopwatch", "stove", "strainer", "tram", "stretcher", "couch", "stupa", "submarine", "suit", "sundial", "sunglasses", "sunglasses", "sunscreen", "suspension bridge", "mop", "sweatshirt", "swim trunks / shorts", "swing", "electrical switch", "syringe", "table lamp", "tank", "tape player", "teapot", "teddy bear", "television", "tennis ball", "thatched roof", "front curtain", "thimble", "threshing machine", "throne", "tile roof", "toaster", "tobacco shop", "toilet seat", "torch", "totem pole", "tow truck", "toy store", "tractor", "semi-trailer truck", "tray", "trench coat", "tricycle", "trimaran", "tripod", "triumphal arch", "trolleybus", "trombone", "hot tub", "turnstile", "typewriter keyboard", "umbrella", "unicycle", "upright piano", "vacuum cleaner", "vase", "vaulted or arched ceiling", "velvet fabric", "vending machine", "vestment", "viaduct", "violin", "volleyball", "waffle iron", "wall clock", "wallet", "wardrobe", "military aircraft", "sink", "washing machine", "water bottle", "water jug", "water tower", "whiskey jug", "whistle", "hair wig", "window screen", "window shade", "Windsor tie", "wine bottle", "airplane wing", "wok", "wooden spoon", "wool", "split-rail fence", "shipwreck", "sailboat", "yurt", "website", "comic book", "crossword", "traffic or street sign", "traffic light", "dust jacket", "menu", "plate", "guacamole", "consomme", "hot pot", "trifle", "ice cream", "popsicle", "baguette", "bagel", "pretzel", "cheeseburger", "hot dog", "mashed potatoes", "cabbage", "broccoli", "cauliflower", "zucchini", "spaghetti squash", "acorn squash", "butternut squash", "cucumber", "artichoke", "bell pepper", "cardoon", "mushroom", "Granny Smith apple", "strawberry", "orange", "lemon", "fig", "pineapple", "banana", "jackfruit", "cherimoya (custard apple)", "pomegranate", "hay", "carbonara", "chocolate syrup", "dough", "meatloaf", "pizza", "pot pie", "burrito", "red wine", "espresso", "tea cup", "eggnog", "mountain", "bubble", "cliff", "coral reef", "geyser", "lakeshore", "promontory", "sandbar", "beach", "valley", "volcano", "baseball player", "bridegroom", "scuba diver", "rapeseed", "daisy", "yellow lady's slipper", "corn", "acorn", "rose hip", "horse chestnut seed", "coral fungus", "agaric", "gyromitra", "stinkhorn mushroom", "earth star fungus", "hen of the woods mushroom", "bolete", "corn cob", "toilet paper"]
# imagenet_classes = [
#     "dog", "canine","doggy",'doggie','hound','pooch','tyke',
#     'cat','feline','house cat','kitty','moggy']

# imagenet_templates = [
#     'petsarefamily {}',
#     'petit {}',
#     'petstore {}',
#     'petfluencer {}',
#     'petfancy {}',
#     'shelterdogs {}',
#     'puppie {}',
#     'fortheloveofpets {}',
#     'adoptapet {}',
#     'rescue {} rule',
#     'ilovepets {}',
#     'petsofinstagram {}',
#     'adoptdontshop {}',
#     'instapet {}',
#     'petsagram {}',
#     'catsagram {}',
#     'petlovers {}',
#     'petshop {}',
#     'adopt {}',
#     'catlover {}',
#     'petstagram {}',
#     'instacat {}',
#     'a plushie {}.',
#     'the plushie {}.',
#     'a cute {} plushie.',
#     'a strong {} plushie.',
#     'a newborn {}.',
#     'an old {}.',
#     'a cute {}.',
#     'an ugly {}.',
#     'a nice {}.',
#     'a bad {}.',
#     'a hurt {}.',
#     'a weak {}.',
#     'a smart {}.',
#     'a good looking {}.',
#     'a bad looking {}.',
#     'a sick {}.',
#     'a inteligent {}.',
#     'a sad {}.',
#     'a strong {}.',
#     'a hungry {}.',
#     'an emoji with a {}.',
#     'an icon with a {}.',
#     'an image of a {} to click.',
#     'a happy {}.',
#     'a photo of the back of a {}.',
#     'the back of a {}.',
#     'a {} to adopt.',
#     'a {} in a jail.',
#     'a {} in a garden.',
#     'a {} running.',
#     'a {} sleeping.',
#     'a {} smiling.',
#     'a {} with blue eyes.',
#     'a {} with grey eyes.',
#     'a {} with black eyes.',
#     'a {} with light eyes.',
#     'a {} with green eyes.',
#     'a {} with yellow eyes.',
#     'a {} with red eyes.',
#     'a close up of a {}',
#     'a laying down {}',
#     'a bad photo of a {}.',
#     'a photo of many {}.',
#     'a sculpture of a {}.',
#     'a {} with text.',
#     'a photo of the hard to see {}.',
#     'a low resolution photo of the {}.',
#     'a rendering of a {}.',
#     'graffiti of a {}.',
#     'a bad photo of the {}.',
#     'a cropped photo of the {}.',
#     'a tattoo of a {}.',
#     'the embroidered {}.',
#     'a photo of a hard to see {}.',
#     'a bright photo of a {}.',
#     'a photo of a clean {}.',
#     'a photo of a light color {}.',
#     'a photo of a dark color {}.',
#     'a photo of a white color {}.',
#     'a photo of a black color {}.',
#     'a photo of a gray color {}.',
#     'a dark photo of the {}.',
#     'a drawing of a {}.',
#     'a photo of my {}.',
#     'the plastic {}.',
#     'a photo of the cool {}.',
#     'a close-up photo of a {}.',
#     'a black and white photo of the {}.',
#     'a painting of the {}.',
#     'a painting of a {}.',
#     'a pixelated photo of the {}.',
#     'a bright photo of the {}.',
#     'a cropped photo of a {}.',
#     'a plastic {}.',
#     'a jpeg corrupted photo of a {}.',
#     'a blurry photo of the {}.',
#     'a photo of the {}.',
#     'a good photo of the {}.',
#     'a rendering of the {}.',
#     'a {} in a video game.',
#     'a photo of one {}.',
#     'a doodle of a {}.',
#     'a close-up photo of the {}.',
#     'a photo of a {}.',
#     'the {} in a video game.',
#     'a sketch of a {}.',
#     'a doodle of the {}.',
#     'a origami {}.',
#     'a low resolution photo of a {}.',
#     'the toy {}.',
#     'a rendition of the {}.',
#     'a photo of the clean {}.',
#     'a photo of a large {}.',
#     'a photo of a nice {}.',
#     'a photo of a weird {}.',
#     'a blurry photo of a {}.',
#     'a cartoon {}.',
#     'art of a {}.',
#     'a sketch of the {}.',
#     'a embroidered {}.',
#     'a pixelated photo of a {}.',
#     'itap of the {}.',
#     'a jpeg corrupted photo of the {}.',
#     'a good photo of a {}.',
#     'a photo of the nice {}.',
#     'a photo of the small {}.',
#     'a photo of the weird {}.',
#     'the cartoon {}.',
#     'art of the {}.',
#     'a drawing of the {}.',
#     'a photo of the large {}.',
#     'a black and white photo of a {}.',
#     'a dark photo of a {}.',
#     'itap of a {}.',
#     'a toy {}.',
#     'itap of my {}.',
#     'a photo of a cool {}.',
#     'a photo of a small {}.',
#     'a thin {}.',
#     'a fat {}.',
#     'a gorgeous {}.',
#     'a popular {}.',
#     'a {} with eye contact.',
#     'eye contact {}.',
#     'a big head {}.',
#     'an interesting {}.',
# ]

# print(f"{len(imagenet_classes)} classes, {len(imagenet_templates)} templates")

In [ ]:
# def zeroshot_classifier(classnames, templates):
#     with torch.no_grad():
#         zeroshot_weights0 = []
#         zeroshot_weights1 = []
#         for classname in tqdm(classnames):
#             texts = [template.format(classname) for template in templates] #format with class
#             texts = clip.tokenize(texts).cuda() #tokenize
#             class_embeddings0 = model0.encode_text(texts) #embed with text encoder
#             class_embeddings0 /= class_embeddings0.norm(dim=-1, keepdim=True)
#             class_embeddings1 = model1.encode_text(texts) #embed with text encoder
#             class_embeddings1 /= class_embeddings1.norm(dim=-1, keepdim=True)
#             #class_embedding = class_embeddings.mean(dim=0)
#             #class_embedding /= class_embedding.norm()
#             print(class_embeddings0.shape, class_embeddings1.shape)
#             zeroshot_weights0.append(class_embeddings0)
#             zeroshot_weights1.append(class_embeddings1)
#         zeroshot_weights0 = torch.cat(zeroshot_weights0, dim=0).cuda()
#         zeroshot_weights1 = torch.cat(zeroshot_weights1, dim=0).cuda()
#     return zeroshot_weights0, zeroshot_weights1

# zeroshot_weights0, zeroshot_weights1 = zeroshot_classifier(imagenet_classes, imagenet_templates)
# zeroshot_weights0.shape, zeroshot_weights1.shape

In [ ]:
# len(imagenet_classes), len(imagenet_templates)

In [ ]:
# LABELS = []
# for classname in imagenet_classes:
#     texts = [template.format(classname) for template in imagenet_templates] #format with class
#     LABELS.extend(texts) 
# len(LABELS), len(LABELS[0])

In [ ]:
# import glob
# files = glob.glob('../input/petfinder-pawpularity-score/train/*.jpg')
# len(files), files[0]

In [ ]:
# image = Image.open(files[146]).convert("RGB")
# image

In [ ]:
# image0 = preprocess0(image).unsqueeze(0)
# image1 = preprocess1(image).unsqueeze(0)
# image_features0 = model0.encode_image(image0.to('cuda'))
# image_features1 = model1.encode_image(image1.to('cuda'))

# image_features0 /= image_features0.norm(dim=-1, keepdim=True)
# image_features1 /= image_features1.norm(dim=-1, keepdim=True)

# logits0 = image_features0 @ zeroshot_weights0.T
# logits1 = image_features1 @ zeroshot_weights1.T
# print( image_features0.shape, logits0.shape, logits1.shape )

# logits0 = logits0.detach().cpu().numpy().flatten()
# logits1 = logits1.detach().cpu().numpy().flatten()

# # if np.mean(logits[:854])>=np.mean(logits[854:]):
# #     logits[854:] = 0
# # else:
# #     logits[:854] = 0
    
# plt.plot( logits0 )
# plt.plot( logits1 )

In [ ]:
# for l in np.argsort(-logits0-logits1)[:50]:
#     print(LABELS[l])

In [ ]:
# class CustomDataset:
#     def __init__(self, data, base_path='../input/petfinder-pawpularity-score/train/', preprocess=None):
        
#         self.data = data.copy()
#         self.base_path = base_path
#         if 'Pawpularity' not in self.data.columns:
#             self.data['Pawpularity'] = 0
#         self.preprocess=preprocess
        
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, item):
#         img = Image.open(self.base_path + self.data.Id[item] + '.jpg').convert("RGB")
#         img = self.preprocess(img)
#         return img

# train_dataset = CustomDataset(data = train, base_path='../input/petfinder-pawpularity-score/train/', preprocess=preprocess0)
# len(train_dataset)

In [ ]:
# model0.cuda().eval()
# train_dataset = CustomDataset(data = train, base_path='../input/petfinder-pawpularity-score/train/', preprocess=preprocess0)
# EMB0 = []
# with torch.no_grad():
#     train_data_loader = DataLoader(train_dataset, batch_size=64,num_workers=2,shuffle=False,pin_memory=True,)
#     for batch in tqdm(train_data_loader):
#         image_features = model0.encode_image(batch.to('cuda'))
#         image_features /= image_features.norm(dim=-1, keepdim=True)
#         #logits = image_features @ zeroshot_weights0.T
#         #logits = logits.cpu().numpy()
#         logits = image_features.cpu().numpy()
#         EMB0.append(logits)
# EMB0 = np.concatenate(EMB0, 0)
# EMB0 = EMB0.astype('float32')
# gc.collect()


# model1.cuda().eval()
# train_dataset = CustomDataset(data = train, base_path='../input/petfinder-pawpularity-score/train/', preprocess=preprocess1)
# EMB1 = []
# with torch.no_grad():
#     train_data_loader = DataLoader(train_dataset, batch_size=64,num_workers=2,shuffle=False,pin_memory=True,)
#     for batch in tqdm(train_data_loader):
#         image_features = model1.encode_image(batch.to('cuda'))
#         image_features /= image_features.norm(dim=-1, keepdim=True)
#         #logits = image_features @ zeroshot_weights1.T
#         #logits = logits.cpu().numpy()
#         logits = image_features.cpu().numpy()
#         EMB1.append(logits)
# EMB1 = np.concatenate(EMB1, 0)
# EMB1 = EMB1.astype('float32')
# gc.collect()

# # flag0 = np.mean(EMB[:,:854],1)
# # flag1 = np.mean(EMB[:,854:],1)
# # EMB[flag0>=flag1, 854:] = 0.
# # EMB[flag0 <flag1, :854] = 0.

# EMB0.shape, EMB1.shape

In [ ]:
# EMB = np.hstack((EMB0, EMB1))
# EMB.shape

In [ ]:
# plt.plot(EMB[0])
# plt.plot(EMB[1])
# plt.plot(EMB[2])
# plt.plot(EMB[3])
# plt.plot(EMB[4])

In [ ]:
# del model0, model1, train_data_loader, train_dataset, zeroshot_weights0, zeroshot_weights1
# gc.collect()
# torch.cuda.empty_cache() # PyTorch thing

In [ ]:
# resd = np.zeros( (EMB.shape[0], len(imagenet_templates)) )
# for ind in range(0, 7*len(imagenet_templates), len(imagenet_templates)):
#     print(ind)
#     resd += EMB[:,ind:ind+len(imagenet_templates)]
# resd /= len(imagenet_templates)
# resd.shape

In [ ]:
# resd = np.zeros( (EMB.shape[0], len(imagenet_templates)) )
# for ind in range(0, 7*len(imagenet_templates), len(imagenet_templates)):
#     print(ind)
#     resd += EMB[:,ind:ind+len(imagenet_templates)]
# resd /= 7
# resd.shape

In [ ]:
# B = 7*len(imagenet_templates)
# resc = np.zeros( (EMB.shape[0], len(imagenet_templates)) )
# for ind in range(0, 5*len(imagenet_templates), len(imagenet_templates)):
#     print(B+ind)
#     resc += EMB[:,ind+B:ind+len(imagenet_templates)+B]
# resc /= 5
# resc.shape

In [ ]:
# plt.plot( resd[5] )
# plt.plot( resc[5] )

In [ ]:
# import cuml
# from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer
# from sklearn.ensemble import RandomForestRegressor
# from cuml.linear_model import LinearRegression
# from cuml.svm import SVR

# # scaler = StandardScaler()
# # EMB = scaler.fit_transform(EMB)
# # gc.collect()

In [ ]:
# ypred = np.zeros(train.shape[0])
# for fold in range(train.fold.max()+1):
#     ind_train = train.fold != fold
#     ind_valid = train.fold == fold
#     model = SVR(C=16.0, kernel='rbf', degree=3, max_iter=4000, output_type='numpy')
#     model.fit(resd[ind_train], train.Pawpularity[ind_train].clip(1, 100)  )
#     ypred[ind_valid] = np.clip(model.predict(resd[ind_valid]), 1 , 100)
#     print( fold, np.sqrt( np.mean( (train.Pawpularity[ind_valid]-ypred[ind_valid])**2 ) ) )
#     del model
#     gc.collect()

# print(np.sqrt(np.mean( (train.Pawpularity-ypred)**2)))

In [ ]:
# ypred = np.zeros(train.shape[0])
# for fold in range(train.fold.max()+1):
#     ind_train = train.fold != fold
#     ind_valid = train.fold == fold
#     model = SVR(C=16.0, kernel='rbf', degree=3, max_iter=4000, output_type='numpy')
#     model.fit(resc[ind_train], train.Pawpularity[ind_train].clip(1, 100)  )
#     ypred[ind_valid] = np.clip(model.predict(resc[ind_valid]), 1 , 100)
#     print( fold, np.sqrt( np.mean( (train.Pawpularity[ind_valid]-ypred[ind_valid])**2 ) ) )
#     del model
#     gc.collect()

# print(np.sqrt(np.mean( (train.Pawpularity-ypred)**2)))

In [ ]:
# resdc = np.hstack((resd, resc))
# scaler = StandardScaler()
# resdc = scaler.fit_transform(resdc)

# ypred = np.zeros(train.shape[0])
# for fold in range(train.fold.max()+1):
#     ind_train = train.fold != fold
#     ind_valid = train.fold == fold
#     model = SVR(C=20.0, kernel='rbf', degree=3, max_iter=4000, output_type='numpy')
#     model.fit(resdc[ind_train], train.Pawpularity[ind_train].clip(1, 100)  )
#     ypred[ind_valid] = np.clip(model.predict(resdc[ind_valid]), 1 , 100)
#     print( fold, np.sqrt( np.mean( (train.Pawpularity[ind_valid]-ypred[ind_valid])**2 ) ) )
#     del model
#     gc.collect()

# print(np.sqrt(np.mean( (train.Pawpularity-ypred)**2)))

In [ ]:
# indd = np.mean(EMB[:,:847],1)>np.mean(EMB[:,847:], 1)
# indc = np.mean(EMB[:,:847],1)<np.mean(EMB[:,847:], 1)

# for i in range(len(imagenet_templates)):
#     print(i, np.corrcoef(train.Pawpularity.values[indd], resc[indd,i])[0,1], np.corrcoef(train.Pawpularity.values[indc], resc[indc,i])[0,1], imagenet_templates[i])

In [ ]:
# import joblib
# RES = joblib.load('../input/petfinderdata/train-embeddings-1.joblib')
# names = ['adv_inception_v3', 'deit_base_patch16_224', 'dla34', 'dm_nfnet_f2', 'fbnetc_100', 'hrnet_w30', 'mixnet_m', 'nfnet_l0', 'pit_ti_distilled_224', 'repvgg_b0', 'tf_mixnet_s', 'twins_pcpvt_small', 'cait_xs24_384', 'deit_base_distilled_patch16_384', 'vit_large_r50_s32_384', 'xception71', 'tf_efficientnetv2_l_in21ft1k', 'tf_efficientnet_b6_ns', 'ig_resnext101_32x8d', 'tf_efficientnet_b5_ap', 'vit_large_patch16_224', 'resnetv2_101x1_bitm', 'resnest200e', 'swsl_resnext101_32x4d', 'seresnet50', 'resnet34d', 'ssl_resnext50_32x4d', 'dpn68', 'hrnet_w18', 'ese_vovnet19b_dw']
# names = [
#  'fbnetc_100',
#  'deit_base_patch16_224',
#  'dm_nfnet_f2',
#  'nfnet_l0',
#  'cait_xs24_384',
#  'deit_base_distilled_patch16_384',
#  'vit_large_r50_s32_384',
#  'tf_efficientnetv2_l_in21ft1k',
#  'tf_efficientnet_b6_ns',
#  'ig_resnext101_32x8d',
#  'tf_efficientnet_b5_ap',
#  'vit_large_patch16_224',
#  'resnetv2_101x1_bitm',
#  'swsl_resnext101_32x4d',
#  'seresnet50',
#  'ssl_resnext50_32x4d']
# len(RES), RES.keys()

In [ ]:
# TRAIN = np.concatenate([RES[k] for k in names], 1)
# gc.collect()
# TRAIN.shape

In [ ]:
# TRAIN = np.concatenate([TRAIN, EMB], 1)

# scaler = StandardScaler()
# TRAIN = scaler.fit_transform(TRAIN)
# gc.collect()
# TRAIN.shape

In [ ]:
# ypred = np.zeros(train.shape[0])
# for fold in range(train.fold.max()+1):
#     ind_train = train.fold != fold
#     ind_valid = train.fold == fold
#     model = SVR(C=16.0, kernel='rbf', degree=4, max_iter=4000, output_type='numpy')
    
#     model.fit(TRAIN[ind_train], train.Pawpularity[ind_train].clip(10, 80)  )
    
#     ypred[ind_valid] = np.clip(model.predict(TRAIN[ind_valid]), 1 , 100)
#     print( fold, np.sqrt( np.mean( (train.Pawpularity[ind_valid]-ypred[ind_valid])**2 ) ) )
#     del model
#     gc.collect()

# print(np.sqrt(np.mean( (train.Pawpularity-ypred)**2)))

In [ ]:
# print(np.sqrt(np.mean( (train.Pawpularity-ypred*1.03)**2)))
#17.23

In [ ]:
# ypred = np.zeros(train.shape[0])
# for fold in range(train.fold.max()+1):
#     ind_train = train.fold != fold
#     ind_valid = train.fold == fold
#     model = SVR(C=15.0, kernel='rbf', degree=3, max_iter=4000, output_type='numpy')
#     model.fit(TRAIN[ind_train,:16000], train.Pawpularity[ind_train].clip(1, 100)  )
#     ypred[ind_valid] = np.clip(model.predict(TRAIN[ind_valid,:16000]), 1 , 100)
#     print( fold, np.sqrt( np.mean( (train.Pawpularity[ind_valid]-ypred[ind_valid])**2 ) ) )
#     del model
#     gc.collect()

# print(np.sqrt(np.mean( (train.Pawpularity-ypred)**2)))

In [ ]:
# ypred = np.zeros(train.shape[0])
# for fold in range(train.fold.max()+1):
#     ind_train = train.fold != fold
#     ind_valid = train.fold == fold
#     model = SVR(C=15.0, kernel='rbf', degree=3, max_iter=4000, output_type='numpy')
#     model.fit(TRAIN[ind_train,16000:], train.Pawpularity[ind_train].clip(1, 100)  )
#     ypred[ind_valid] = np.clip(model.predict(TRAIN[ind_valid,16000:]), 1 , 100)
#     print( fold, np.sqrt( np.mean( (train.Pawpularity[ind_valid]-ypred[ind_valid])**2 ) ) )
#     del model
#     gc.collect()

# print(np.sqrt(np.mean( (train.Pawpularity-ypred)**2)))
# 0 18.390877473712205
# 1 18.37520943999738
# 2 17.911378200162932
# 3 18.360758697517085
# 4 18.09363635381684
# 5 17.65013271217901
# 6 17.88182347957293
# 7 18.114255118805936
# 8 18.5835748512917
# 9 18.313394763931548
# 18.169605838333613
# add Code

# 0 18.352086252886124
# 1 18.281019802072283
# 2 17.999595314064617
# 3 18.188194303577678
# 4 18.018147095683684
# 5 17.65765853422346
# 6 17.659725807423907
# 7 17.886643341551178
# 8 18.30650681757382
# 9 18.285400132808327
# 18.06526378669336

# 0 18.238363195172568
# 1 18.20530498421699
# 2 17.837698742776187
# 3 17.876089048979846
# 4 17.991753075578142
# 5 17.69875706552218
# 6 17.740801963575866
# 7 18.02232325620849
# 8 18.13142870578993
# 9 17.91583399210565
# 17.966755112294212

# 0 18.170296592594394
# 1 18.248313463683786
# 2 17.81405938485103
# 3 18.039395730339145
# 4 17.897243800605775
# 5 17.5610982379652
# 6 17.6048631176059
# 7 17.85875425760877
# 8 18.120320401664348
# 9 18.036662575808865
# 17.93649369609269